In [5]:
import sys
import os
import json

# Jupyter用のパス設定
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

from src.schema.definitions import OutlookConfig
from src.adapter.outlook import OutlookAdapter
from src.engine.core import GenericEtlEngine

# ロジックカタログのロード (これをしないと機能が登録されない)
import src.catalog.handlers.basic
import src.catalog.handlers.document
import src.catalog.handlers.archive

def run_from_external_json(config_file_path):
    # -------------------------------------------------
    # 1. 外部JSONの読み込み (Load)
    # -------------------------------------------------
    if not os.path.exists(config_file_path):
        print(f"❌ ファイルが見つかりません: {config_file_path}")
        return

    print(f"📂 設定ファイルをロード中...: {config_file_path}")
    
    with open(config_file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
        # ここで Pydantic が「仕様通りか？」を厳密にチェック
        try:
            config = OutlookConfig(**data)
            print("✅ 設定ファイルのバリデーションOK")
        except Exception as e:
            print(f"❌ 設定ファイルの内容が不正です: {e}")
            return

    # -------------------------------------------------
    # 2. エンジンの起動 (Run)
    # -------------------------------------------------
    print(f"🤖 ジョブ起動: {config.job_name}")
    print(f"   検索ワード: {config.search_keywords}")
    
    try:
        # アダプターとエンジンを結合
        adapter = OutlookAdapter()
        engine = GenericEtlEngine(config, adapter)
        
        # 実行！
        engine.run()
        print("✅ 全処理が完了しました")
        
    except Exception as e:
        print(f"💥 実行中にエラーが発生しました: {e}")
        # Outlookが入っていない環境等の場合、ここでモック動作に切り替える判断も可能

# --- 実行 ---
# Step 1で作ったファイルを指定して実行
run_from_external_json("configs/manual_test.json")

📂 設定ファイルをロード中...: configs/manual_test.json
✅ 設定ファイルのバリデーションOK
🤖 ジョブ起動: Manual_JSON_Test_Job
   検索ワード: ['請求書', 'TestEmail']
>> [Adapter] Outlookに接続しました
>> [Adapter] 検索 '請求書': 3 件ヒット
  - メール発見: Microsoft 請求書 G128449371 の準備ができました
    [OCR ] G128449371.pdf をテキスト化しました -> G128449371.pdf.txt
  - メール発見: Microsoft 請求書 G133796987 の準備ができました
    [OCR ] G133796987.pdf をテキスト化しました -> G133796987.pdf.txt
  - メール発見: テスト用請求書
    [保存] test.xlsx を保存しました。
>> [Adapter] 検索 'TestEmail': 0 件ヒット
✅ 全処理が完了しました
